# Modern Data Science 
**(Module 01: A Touch of Data Science)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.

Prepared by and for 
**Student Members** |
2006-2018 [TULIP Lab](http://www.tulip.org.au), Australia

---


# Session C - Association Rule Mining

Import the package

In [ ]:
from numpy import *

Creat a simple data set to test

In [ ]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

The function  createC1() creates—you guessed it— C1 .  C1 is a candidate itemset of size one.
In the Apriori algorithm, we create  C1 , and then we’ll scan the dataset to see
if these one itemsets meet our minimum support requirements. The itemsets that do
meet our minimum requirements become  L1 .  L1 then gets combined to become  C2
and  C2 will get filtered to become  L2 . This is the main idea of this lgorithm.

In [ ]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#use frozen set so we
                            #can use it as a key in a dict 

This function(scanD) takes three arguments: a dataset,  Ck , a list of candidate sets, and  minSupport , which is the minimum
support you’re interested in. This is the function you’ll use to generate  L1 from  C1.

In [ ]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt.keys(): ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(list(D)))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [ ]:
# !pip install apriori
!pip install apyori

In [ ]:
# import apriori
from apyori import apriori

In [ ]:
dataSet = loadDataSet()
print(dataSet)

In [ ]:
C1 = createC1(dataSet)
# for i in list(C1):
#     print(i)
lisC=list(C1)
print(lisC)

In [ ]:
D = map(set, dataSet)
lisD=list(D)
print(lisD)
print(type(lisD[0]))
print (len(lisD))
for i in lisD:
    print(i)

# print(type(list(D)[0]))

In [ ]:
L1,suppData0 = scanD(lisD, lisC, 0.5)
print (L1)

The function  aprioriGen() will take a list of frequent itemsets, Lk , and the size of the itemsets,  k , to produce  Ck . For example, it will take the itemsets {0}, {1}, {2} and so on and produce {0,1} {0,2}, and {1,2}.

In [ ]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [ ]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    L1, supportData = scanD(lisD, lisC, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(lisD, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [ ]:
L,suppData = apriori(dataSet)
print (L)

In [ ]:
L[0]

In [ ]:
L[1]

In [ ]:
L[2]

In [ ]:
L[3]

In [ ]:
L,suppData = apriori(dataSet, minSupport=0.7)

In [ ]:
L

The function is going to generate a list of rules with confidence values that we can sort through later.

In [ ]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

The function is mainly to evaluate those rules.

In [ ]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

The function is to generate a set of candidate rules.

In [ ]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [ ]:
def pntRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print (itemMeaning[item])
        print ("           -------->")
        for item in ruleTup[1]:
            print (itemMeaning[item])
        print ("confidence: %f" % ruleTup[2])

In [ ]:
L,suppData = apriori(dataSet, minSupport = 0.5)

In [ ]:
rules = generateRules(L, suppData, minConf = 0.7)

rules

In [ ]:
rules = generateRules(L, suppData, minConf = 0.5)

In [ ]:
rules

#### An Example: finding similar features in poisonous mushrooms

In [ ]:
import wget

link_to_data = 'https://github.com/tuliplab/mds/raw/master/Jupyter/data/mushroom.dat'
DataSet = wget.download(link_to_data)

In [ ]:
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]

In [ ]:
L,suppData=apriori(mushDatSet, minSupport=0.3)

In [ ]:
for item in L[1]:
    if item.intersection('2'): 
        print (item)

In [ ]:
for item in L[3]:
    if item.intersection('2'): 
        print (item)

#### Another Example: Market Basket

In [1]:
import wget

link_to_data = 'https://github.com/tuliplab/mds/raw/master/Jupyter/data/Market_Basket_Optimisation.csv'
DataSet = wget.download(link_to_data)

In [ ]:
# !pip install apyori

In [ ]:
import pandas as pd
from apyori import apriori

In [ ]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None)

In [ ]:
transactions = []
for i in range(0,len(dataset)):
    currentItem = dataset.iloc[i,:].values
#    print(currentItem)
    currentTransaction = []
#    print(currentTransaction)
    for j in range(0,len(currentItem)):
        currentTransaction.append(str(currentItem[j]))
#    print(currentTransaction)
    transactions.append(currentTransaction)

In [ ]:
rules = apriori(transactions, min_support = 0.003,min_confidence = 0.2,min_lift = 3,min_length = 2)

In [ ]:
# Show the results
results = list(rules)
print(results)

In [ ]:
for i in range(len(results)):
    print("##############################################################################")
    print(i)
    print(results[i])
    print(results[i].items)
    